# Logistics Hackathon Preparation 

## get started 
- create venv 
```shell 
pip install venv 
python -m venv .env
```

- activate venv  

powershell: 
```shell 
.env/Scripts/Activate.ps1
```

cmd: 
```shell 
env/Scripts/activate.bat
```

- install requirements 
```shell 
pip install -r requirements.txt
```

In [5]:
import networkx as nx 
from pydantic import BaseModel
import json 

## 1. load graph from json into networkx graph object 
- use json module to load graph.json 
- use networkx `add_edge()` function to create the graph with 'distance' on every edge [documentation](https://networkx.org/documentation/stable/reference/classes/generated/networkx.Graph.add_edge.html) 

In [47]:

graph = nx.Graph() 

# load graph 
with open("graph.json", "r") as infile: 
    data = json.load(infile)

# add edges to graph object
for origin in data: 
    for destination in data[origin]: 
        graph.add_edge(origin, destination, distance = data[origin][destination]["distance"])

for node in graph.nodes: 
    graph.add_node(node, observed_revenue = [])

graph["start"]

AtlasView({'fxuhs': {'distance': 262.18}, 'kmftu': {'distance': 331.32}, 'vghbm': {'distance': 323.22}})

## 2. create function that calculates the profit of a route
- 1km costs 1.2€

In [7]:
def get_profit(distance:float, revenue:float) -> float: 
    return revenue - distance*1.2


## 3. search the shortest path distance between two nodes 
- use networkx shortest path length function [documentation](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.shortest_paths.generic.shortest_path_length.html#networkx.algorithms.shortest_paths.generic.shortest_path_length)

In [13]:
def get_distance(origin: str, destination: str) -> float:
    return nx.shortest_path_length(graph, origin, destination, weight = "distance")

get_distance(origin="start", destination="end")

17065.049999999996

## 4. search the shortest path distance between multiple nodes 
- return node idx and distance for the shortest path 
- use networkx all shortest path lenghts function [documentation](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.shortest_paths.unweighted.single_source_shortest_path_length.html#networkx.algorithms.shortest_paths.unweighted.single_source_shortest_path_length)

In [25]:
def get_minimum_distance(origin: str, destinations: list[str]) -> tuple[str, float]:
    paths = nx.shortest_path_length(graph, source = origin, weight = "distance")
    paths = {node:distance for node, distance in paths.items() if node in destinations}
    return sorted(paths.items(), key=lambda x: x[0])[0] 


get_minimum_distance(origin="end", destinations=["end", "tpnkm", "qzelt", "bcwum", "aeiou"])

('bcwum', 2638.0600000000004)

## 5. calculate shortest path and return an Route Object
- put previous functions together and return Route Object

In [29]:
class Route(BaseModel):
    origin: str
    destination: str
    distance: float 
    profit: float 


def get_route_info(origin, destination, revenue) -> Route: 
    distance = get_distance(origin, destination)
    profit = get_profit(distance, revenue)
    return  Route(
        origin=origin, 
        destination=destination, 
        distance=distance, 
        profit=profit
    )

get_route_info(origin="start", destination="end", revenue = 30_000)


Route(origin='start', destination='end', distance=17065.049999999996, profit=9521.940000000006)

## 6. Put everything together in routing.py 

In [32]:
import routing 

route_info = routing.get_route_info(origin="start", destination="end", revenue = 30_000)
route_info

Route(origin='start', destination='end', distance=17065.049999999996, profit=9521.940000000006)

## optional: search nodes within distance < 1000 

In [60]:
import time 
import random 
nodes = list(graph.nodes)

start = time.time() 

for _ in range(1000):
    
    node = random.choice(nodes)
    paths = nx.single_source_dijkstra_path_length(graph, source=node, weight="distance", cutoff=1000)

    for n, distance in paths.items():
        if n != node:
            graph.nodes[n]["observed_revenue"].append(distance)

print(f"runtime: {time.time()-start:.2f}")

runtime: 0.13


In [62]:
graph.nodes[n]["observed_revenue"]

[607.77,
 854.24,
 790.36,
 414.16999999999996,
 958.27,
 598.48,
 831.65,
 857.79,
 422.26,
 855.01,
 166.92000000000002,
 702.8499999999999,
 702.8499999999999,
 41.53,
 351.72,
 506.6,
 842.43,
 738.12,
 738.12,
 842.6899999999999,
 543.78,
 414.16999999999996,
 891.51,
 365.21000000000004,
 992.8499999999999]

In [61]:
import numpy as np 

np.mean(graph.nodes[n]["observed_revenue"])

641.254